In [72]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import sklearn.ensemble as ensemble
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [73]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
path = Path("Resources/lending_data.csv")
lending_df = pd.read_csv(path)
# Review the DataFrame
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


In [74]:
lending_df['loan_status'].value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [75]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = lending_df['loan_status']

# Separate the X variable, the features
X = lending_df.drop(columns='loan_status')

In [76]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [77]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Split the data into training and testing datasets by using `train_test_split`.

In [78]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [79]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [80]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Generate a confusion matrix.

* Print the classification report.

In [81]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
print("Confusion Matrix:")
print("_ _ _ _ _ _ _ _ _ _ ")
cm_df
#tn, fp, fn, tp

Confusion Matrix:
_ _ _ _ _ _ _ _ _ _ 


,Predicted 0,Predicted 1
Actual 0,18658,107
Actual 1,37,582


In [82]:
# Print the classification report for the model
target_names = ["Healthy loans", "High risk loans"]

print("Classification Report:")
print("_ _ _ _ _ _ _ _ _ _ _ _  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ")
print(classification_report(y_test, predictions, target_names=target_names))

Classification Report:
_ _ _ _ _ _ _ _ _ _ _ _  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
                 precision    recall  f1-score   support

  Healthy loans       1.00      0.99      1.00     18765
High risk loans       0.84      0.94      0.89       619

       accuracy                           0.99     19384
      macro avg       0.92      0.97      0.94     19384
   weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model predicts the occurance of healthy loans (0) with a high degree of accuracy, achieving a value of 1 for precision and 0.99 for recall. The prediction for the occurance of high risk loans (1) is still relatively strong but weaker than the prediction of healthy loans, with the classification of high risk loans reporting a precision of 0.84 and a recall of 0.94 respectively. 

Overall, this means that only a small number of people with a high risk of default have been classified incorrectly as low risk (False Negatives), and only a marginally higher number with a low risk have been misclassified as high risk by the prediction (False Positives). 

## Create multiple model dataframe to compare

### 5. Scale the data using `StandardScaler`

In [83]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

### 6. Run multiple ML models and compare results

In [84]:
ml = {
    "KNeighborsClassifier": KNeighborsClassifier(n_neighbors=3),
    "RandomForestClassifier": ensemble.RandomForestClassifier(n_estimators=100, random_state=78),    
    "AdaBoostClassifier": ensemble.AdaBoostClassifier(n_estimators=50, random_state=78),    
    "AdaBoostClassifier2": ensemble.AdaBoostClassifier(n_estimators=100, random_state=78),    
    "BaggingClassifier": ensemble.BaggingClassifier(n_estimators=50, random_state=78),    
    "ExtraTreesClassifier": ensemble.ExtraTreesClassifier(n_estimators=50, random_state=78),    
}
all_data = []
for x in ml:
    model = ml[x]
    model.fit(X_train_scaled, y_train)# Create predictions
    y_pred = model.predict(X_test_scaled)
    tn, fp, fn, tp = confusion_matrix(y_pred,y_test).ravel()
    report = classification_report(y_pred,y_test, output_dict=True)
    d = {
        "ml":x,
        "score": model.score(X_test_scaled,y_test),
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "tp": tp,
    }
    all_data.append({**d, **report})
    
sorted_df = pd.DataFrame(all_data).sort_values("score")
sorted_df

,ml,score,tn,fp,fn,tp,0,1,accuracy,macro avg,weighted avg
5,ExtraTreesClassifier,0.990817,18667,80,98,539,"{'precision': 0.9947775113242739, 'recall': 0....","{'precision': 0.8707592891760905, 'recall': 0....",0.990817,"{'precision': 0.9327684002501822, 'recall': 0....","{'precision': 0.9907020053653185, 'recall': 0...."
4,BaggingClassifier,0.991488,18666,66,99,553,"{'precision': 0.9947242206235012, 'recall': 0....","{'precision': 0.8933764135702746, 'recall': 0....",0.991488,"{'precision': 0.9440503170968879, 'recall': 0....","{'precision': 0.9913152869566263, 'recall': 0...."
1,RandomForestClassifier,0.991539,18665,64,100,555,"{'precision': 0.9946709299227284, 'recall': 0....","{'precision': 0.8966074313408724, 'recall': 0....",0.991539,"{'precision': 0.9456391806318004, 'recall': 0....","{'precision': 0.9913572902420064, 'recall': 0...."
0,KNeighborsClassifier,0.992158,18657,44,108,575,"{'precision': 0.9942446043165467, 'recall': 0....","{'precision': 0.9289176090468497, 'recall': 0....",0.992158,"{'precision': 0.9615811066816982, 'recall': 0....","{'precision': 0.9919427915963029, 'recall': 0...."
2,AdaBoostClassifier,0.993861,18650,4,115,615,"{'precision': 0.9938715694111377, 'recall': 0....","{'precision': 0.9935379644588045, 'recall': 0....",0.993861,"{'precision': 0.9937047669349711, 'recall': 0....","{'precision': 0.9938590058734157, 'recall': 0...."
3,AdaBoostClassifier2,0.993861,18650,4,115,615,"{'precision': 0.9938715694111377, 'recall': 0....","{'precision': 0.9935379644588045, 'recall': 0....",0.993861,"{'precision': 0.9937047669349711, 'recall': 0....","{'precision': 0.9938590058734157, 'recall': 0...."


In [85]:
for index, row in sorted_df.iterrows():
#     print(f'Model: {row["ml"]}, Accuracy: {row["accuracy"]:.3f}, tn: {row["tn"]}, fp: {row["fp"]}, fn: {row["fn"]}, tp: {row["tp"]}')
    print(f'Model: {row["ml"]}, Score: {row["score"]:.3f}, "0" PRE: {row["0"]["precision"]:.3f}, "0" RE: {row["0"]["recall"]:.3f}, "1" PRE: {row["1"]["precision"]:.3f}, "1" RE: {row["1"]["recall"]:.3f}')

Model: ExtraTreesClassifier, Score: 0.991, "0" PRE: 0.995, "0" RE: 0.996, "1" PRE: 0.871, "1" RE: 0.846
Model: BaggingClassifier, Score: 0.991, "0" PRE: 0.995, "0" RE: 0.996, "1" PRE: 0.893, "1" RE: 0.848
Model: RandomForestClassifier, Score: 0.992, "0" PRE: 0.995, "0" RE: 0.997, "1" PRE: 0.897, "1" RE: 0.847
Model: KNeighborsClassifier, Score: 0.992, "0" PRE: 0.994, "0" RE: 0.998, "1" PRE: 0.929, "1" RE: 0.842
Model: AdaBoostClassifier, Score: 0.994, "0" PRE: 0.994, "0" RE: 1.000, "1" PRE: 0.994, "1" RE: 0.842
Model: AdaBoostClassifier2, Score: 0.994, "0" PRE: 0.994, "0" RE: 1.000, "1" PRE: 0.994, "1" RE: 0.842


---